In [14]:
import os
import os.path
import glob
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold                                                                                                                       
from sklearn.metrics import confusion_matrix,accuracy_score
from sklearn import svm
from sklearn.model_selection import GridSearchCV
import sklearn_evaluation.plot as skplot
from sklearn.model_selection import learning_curve

import matplotlib.pyplot as plt

import numpy as np
np.random.seed(1)

from keras.preprocessing.image import img_to_array
from keras.utils import np_utils
from keras.preprocessing import image
from keras.applications.vgg16 import VGG16
from keras.applications.imagenet_utils import preprocess_input

os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [2]:
imagedir = "Datasets/Detection/Byteplots"

In [3]:
cur_dir = os.getcwd()
os.chdir(imagedir)  # the parent folder with sub-folders

# Get number of samples per family
list_fams = sorted(os.listdir(os.getcwd()), key=str.lower)  # vector of strings with family names
no_imgs = []  # No. of samples per family
for i in range(len(list_fams)):
    os.chdir(list_fams[i])
    len1 = len(glob.glob('*.png'))  # assuming the images are stored as 'png'
    no_imgs.append(len1)
    os.chdir('..')
num_samples = np.sum(no_imgs)  # total number of all samples

# Compute the labels
y = np.zeros(num_samples)
pos = 0
label = 0
for i in no_imgs:
    print ("Label:%2d\tFamily: %15s\tNumber of images: %d" % (label, list_fams[label], i))
    for j in range(i):
        y[pos] = label
        pos += 1
    label += 1
num_classes = label

# Compute the features
width, height,channels = (224,224,3)
X = np.zeros((num_samples, width, height, channels))
cnt = 0
list_paths = [] # List of image paths
print("Processing images ...")
for i in range(len(list_fams)):
    for img_file in glob.glob(list_fams[i]+'/*.png'):
        #print("[%d] Processing image: %s" % (cnt, img_file))
        list_paths.append(os.path.join(os.getcwd(),img_file))
        img = image.load_img(img_file, target_size=(224, 224))
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)
        X[cnt] = x
        cnt += 1
print("Images processed: %d" %(cnt))

os.chdir(cur_dir)

Label: 0	Family:          Banker	Number of images: 1986
Label: 1	Family:          WinExe	Number of images: 1945
Processing images ...


/usr/local/lib/python2.7/dist-packages/PIL/Image.py:2371: DecompressionBombWarning: Image size (103265280 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning)
/usr/local/lib/python2.7/dist-packages/PIL/Image.py:2371: DecompressionBombWarning: Image size (110601216 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning)


Images processed: 3931


In [4]:
X.shape

(3931, 224, 224, 3)

In [5]:
# Creating base_model
image_shape = (224, 224, 3)                                                                                                                                                                                                                                                                                            
base_model = VGG16(weights='imagenet', input_shape=image_shape, include_top=False, pooling='avg')

In [6]:
print("Extracting bottleneck features from convolutional layers ...")
bnfeatures = base_model.predict(X)

Extracting bottleneck features from convolutional layers ...


In [7]:
bnfeatures.shape

(3931, 512)

In [8]:
# Create stratified k-fold subsets                                                                                                                                        
kfold = 5  # no. of folds                                                                 
skf = StratifiedKFold(kfold, shuffle=True,random_state=1)

In [9]:
# Training top_model
C_range = np.logspace(-2, 10, 13)
param_grid = dict(C=C_range)

top_model = GridSearchCV(svm.LinearSVC(), param_grid=param_grid, cv=skf, verbose=10, n_jobs=-1)
top_model.fit(bnfeatures,y)  # Training 

Fitting 5 folds for each of 13 candidates, totalling 65 fits
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.1 ...........................................................
[CV] C=0.1 ...........................................................
[CV] C=0.1 ...........................................................
[CV] C=0.1 ...........................................................
[CV] C=0.1 ...........................................................
[CV] C=1.0 ...........................................................
[CV] C=1.0 ...........................................................
[CV] C=1.0 ...........................................................
[CV] C=1.0 .....

[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    2.1s


[CV] .......................... C=100.0, score=0.879135, total=   1.8s
[CV] C=1000.0 ........................................................
[CV] .......................... C=100.0, score=0.936468, total=   2.1s
[CV] C=1000.0 ........................................................
[CV] ........................... C=0.01, score=0.956798, total=   3.2s
[CV] C=1000.0 ........................................................
[CV] ............................ C=0.1, score=0.928753, total=   3.1s
[CV] C=10000.0 .......................................................
[CV] .......................... C=100.0, score=0.937659, total=   2.1s
[CV] C=10000.0 .......................................................
[CV] ......................... C=1000.0, score=0.932570, total=   2.1s
[CV] ........................... C=0.01, score=0.949109, total=   4.1s
[CV] ........................... C=0.01, score=0.952926, total=   4.2s
[CV] ........................... C=10.0, score=0.936387, total=   3.8s
[CV] C

[Parallel(n_jobs=-1)]: Done  13 tasks      | elapsed:    4.4s


[CV] C=100000.0 ......................................................
[CV] ............................ C=1.0, score=0.930025, total=   4.2s
[CV] ........................... C=10.0, score=0.923664, total=   4.1s
[CV] ........................... C=0.01, score=0.960560, total=   4.3s
[CV] C=1000000.0 .....................................................
[CV] C=100000.0 ......................................................
[CV] C=100000.0 ......................................................
[CV] C=1000000.0 .....................................................
[CV] ......................... C=1000.0, score=0.930114, total=   2.6s
[CV] ............................ C=0.1, score=0.949109, total=   4.5s
[CV] C=1000000.0 .....................................................
[CV] C=1000000.0 .....................................................
[CV] ............................ C=0.1, score=0.940204, total=   4.6s
[CV] C=1000000.0 .....................................................
[CV] .

...

[CV] ................... C=1000000000.0, score=0.939009, total=   2.4s
[CV] ................... C=1000000000.0, score=0.930025, total=   2.6s
[CV] .................. C=10000000000.0, score=0.917303, total=   1.4s
[CV] ................... C=1000000000.0, score=0.917303, total=   2.4s
[CV] ................... C=1000000000.0, score=0.945293, total=   2.6s
[CV] .................. C=10000000000.0, score=0.936468, total=   2.3s
[CV] .................... C=100000000.0, score=0.936387, total=   3.1s
[CV] ................... C=1000000000.0, score=0.926209, total=   2.8s


[Parallel(n_jobs=-1)]: Done  60 out of  65 | elapsed:   10.1s remaining:    0.8s


[CV] .................. C=10000000000.0, score=0.930025, total=   2.4s
[CV] .................. C=10000000000.0, score=0.945293, total=   2.4s
[CV] .................. C=10000000000.0, score=0.936387, total=   2.1s


[Parallel(n_jobs=-1)]: Done  65 out of  65 | elapsed:   10.7s finished


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=1, shuffle=True),
       error_score='raise',
       estimator=LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'C': array([  1.00000e-02,   1.00000e-01,   1.00000e+00,   1.00000e+01,
         1.00000e+02,   1.00000e+03,   1.00000e+04,   1.00000e+05,
         1.00000e+06,   1.00000e+07,   1.00000e+08,   1.00000e+09,
         1.00000e+10])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=10)

In [11]:
top_model.cv_results_

{'mean_fit_time': array([ 3.39932985,  4.06394382,  3.34158854,  4.21187119,  2.92060761,
         2.67562628,  3.33371024,  3.54748144,  3.23897719,  3.44067683,
         2.63901219,  2.54138241,  2.07881279]),
 'mean_score_time': array([ 0.10273619,  0.05745382,  0.0531652 ,  0.04994226,  0.03633022,
         0.06498575,  0.06137156,  0.098206  ,  0.0702486 ,  0.08426538,
         0.04730158,  0.00473275,  0.02175303]),
 'mean_test_score': array([ 0.95319257,  0.94047316,  0.93462223,  0.93563979,  0.92419232,
         0.93462223,  0.9330959 ,  0.93462223,  0.93487662,  0.93462223,
         0.93055202,  0.93156958,  0.9330959 ]),
 'mean_train_score': array([ 0.98925209,  0.99287718,  0.99154171,  0.99268638,  0.97329045,
         0.99122378,  0.99077863,  0.99122378,  0.99141456,  0.99122378,
         0.98486448,  0.97716924,  0.98505526]),
 'param_C': masked_array(data = [0.01 0.10000000000000001 1.0 10.0 100.0 1000.0 10000.0 100000.0 1000000.0
  10000000.0 100000000.0 1000000000.0 

In [10]:
top_model.grid_scores_

/usr/local/lib/python2.7/dist-packages/sklearn/model_selection/_search.py:667: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


[mean: 0.95319, std: 0.00506, params: {'C': 0.01},
 mean: 0.94047, std: 0.00661, params: {'C': 0.10000000000000001},
 mean: 0.93462, std: 0.00691, params: {'C': 1.0},
 mean: 0.93564, std: 0.00662, params: {'C': 10.0},
 mean: 0.92419, std: 0.02309, params: {'C': 100.0},
 mean: 0.93462, std: 0.00560, params: {'C': 1000.0},
 mean: 0.93310, std: 0.00757, params: {'C': 10000.0},
 mean: 0.93462, std: 0.00560, params: {'C': 100000.0},
 mean: 0.93488, std: 0.00603, params: {'C': 1000000.0},
 mean: 0.93462, std: 0.00560, params: {'C': 10000000.0},
 mean: 0.93055, std: 0.00974, params: {'C': 100000000.0},
 mean: 0.93157, std: 0.00978, params: {'C': 1000000000.0},
 mean: 0.93310, std: 0.00927, params: {'C': 10000000000.0}]

In [12]:
print("Best parameters: %s - score: %0.4f" % (top_model.best_params_, top_model.best_score_))

Best parameters: {'C': 0.01} - score: 0.9532
